In [1]:
library(dplyr)
library(magrittr)
library(caret)
library(xgboost)
library(dummies)
library(cattonum)
train=read.csv('C:\\Users\\coco40725\\Documents\\GitHub\\2nd-ML100Days\\data\\titanic_train.csv')
train=read.csv('C:\\Users\\coco40725\\Documents\\GitHub\\2nd-ML100Days\\data\\titanic_train.csv')
train=train[,-c(1,4,9)]

##extract categorical variable
col.type=rep(0,9)
for (i in 1:9) {
  col.type[i]=class(train[,i])
}
col.factor=which(col.type=='factor')



##only keep categorical column
train.cate=train[,c(col.factor,1)]

## covert NA into a level
for (j in 1:3) {
  train.cate[,j] %<>% addNA(.)
  
}

## fill somethong in empty cell
for (d in 1:3) {
  levels(train.cate[,d])[levels(train.cate[,d])=='']='empty'
}

## linear regression+ lable coding
train.cate.label=train.cate
for (k in 1:3) {
  train.cate.label[,k] %<>% as.numeric(.)
}


t11=Sys.time()
train_control <- trainControl(method="cv", number=5)
cv1= train(as.factor(Survived)~., data=train.cate.label, family=binomial,trControl=train_control, method="glm")
t12=Sys.time()
time1=difftime(t11,t12,units='secs')
racc1=cv1$results[2]
rkappa1=cv1$results[3]


# mean coding + regresison
t11=Sys.time()
train.mean=catto_mean(train.cate,c('Sex','Embarked','Cabin'),response=Survived)
cv1.mean= train(as.factor(Survived)~., data=train.mean, family=binomial,trControl=train_control, method="glm")
racc1.mean=cv1.mean$results[2]
rkappa1.mean=cv1.mean$results[3]
t12=Sys.time()
time1.mean=difftime(t11,t12,units='secs')

##hashing +regression
library(FeatureHashing)
f=data.frame(cabin=train.cate$Cabin,sex=train.cate$Sex,emb=train.cate$Embarked)
m1 = model.matrix(~., data=f,hash.size=3,signed.hash=FALSE,create.mapping = TRUE)
t11=Sys.time()
train.hash=data.frame(m1,Survived=train.cate$Survived)
cv1.hash= train(as.factor(Survived)~., data=train.hash, family=binomial,trControl=train_control, method="glm")
t12=Sys.time()
time1.hash=difftime(t11,t12,units='secs')
racc1.hash=cv1.hash$results[2]
rkappa1.hash=cv1.hash$results[3]


###result
result=data.frame(type=c('label coding','mean coding','hashing coding'),
                  acc=unlist(c(racc1,racc1.mean,racc1.hash)),
                  kappa=unlist(c(rkappa1,rkappa1.mean,rkappa1.hash)),
                  time=-c(time1,time1.mean,time1.hash))
result




Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Loading required package: lattice
Loading required package: ggplot2

Attaching package: 'xgboost'

The following object is masked from 'package:dplyr':

    slice



ERROR: Error in library(dummies): there is no package called 'dummies'


#### Mean coding has the best performance